<h1>Understanding the Data</h1>
This notebook reviews the audio attributes available from Spotify and how to retrieve them. I also provide an explanation as to why I chose to utilize the audio feature <b>timbre</b> for this project.

----

<h2>Prerequisites</h2>
<ol>
<li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/1.%20Setting%20Up%20the%20API%20Connection.ipynb'>Setting Up the API Connection</a></li>
<li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/2.%20Using%20the%20Spotify%20API.ipynb'>Using the Spotify API</a></li>
</ol>

---

<h2>Audio Features</h2>
Audio features are attributes of a song such as tempo, loudness, danceability, etc. A breakdown of each of the 13 audio features and their definitions can be found on <a href = 'https://developer.spotify.com/documentation/web-api/reference/get-audio-features'>Spotify's Developer Site</a>. 
This project does not utilize any of the audio features, but I have used them on another project and can say they are a great, fun, and easy way to start to dive into musical attribute analyses.<br>
Even though we are not using them, we will still see how to retrieve the audio features using the Spotipy API.

<h4>Connect to Spotify Using Spotipy</h4>
Let's create a function that will conveniently get us connected to the Spotify API using the method outlined in the <a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/1.%20Setting%20Up%20the%20API%20Connection.ipynb'>Setting up the API Connection</a>.

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def connect_to_spotify():
    sp_client_id = input("Enter your Spotify Client ID: ")
    sp_client_secret = input("Enter your Spotify Secret Token: ")
    
    credentials = SpotifyClientCredentials(client_id = sp_client_id,
                                           client_secret = sp_client_secret)
    
    sp = spotipy.Spotify(client_credentials_manager = credentials)
    
    print("Connected to Spotify!")

    return sp

In [5]:
sp = connect_to_spotify()

Enter your Spotify Client ID:  ed9307841d3542df8819aec9a4f0ec84
Enter your Spotify Secret Token:  8208717955574be6a24163ed59675094


Connected to Spotify!


<h4>Find Track URI</h4>
Using the method outlined in <a href='https://nbviewer.org/github/JonYarber/music_modeling/blob/main/2.%20Using%20the%20Spotify%20API%20.ipynb'>Using the Spotify API</a>, let's create a function that will take an artist and song name and return the track URI.

In [7]:
def get_uri():

    artist = input("Input artist name: ")
    track = input("Input song name: ")

    search_term = f'artist:{artist} track:{track}'

    result = sp.search(search_term, type = 'track', limit = 1)['tracks']['items'][0]

    track_uri = result['uri']
    artist_name = result['artists'][0]['name']
    track_name = result['name']

    print(f'\nTrack name: {track_name}')
    print(f'Artist name: {artist_name}')
    print(f'Track URI: {track_uri}')

    return track_uri

<br>
For this example, we will use the song <i>Karma Police</i> by Radiohead.

In [8]:
track_uri = get_uri()

Input artist name:  Radiohead
Input song name:  Karma Police



Track name: Karma Police
Artist name: Radiohead
Track URI: spotify:track:63OQupATfueTdZMWTxW03A


<h4>Retrieve Audio Features</h4>
Next, we use the track URI to retrieve the audio features. The spotipy object, <b>sp</b>, has a built-in method for this, <b>audio_features</b>.
<br>The only required parameter for <b>audio_features</b> is the track URI.

In [9]:
# Use the track URI to retrieve audio features
audio_features = sp.audio_features(track_uri)

audio_features

[{'danceability': 0.36,
  'energy': 0.501,
  'key': 7,
  'loudness': -9.129,
  'mode': 1,
  'speechiness': 0.0258,
  'acousticness': 0.0638,
  'instrumentalness': 9.32e-05,
  'liveness': 0.172,
  'valence': 0.324,
  'tempo': 74.807,
  'type': 'audio_features',
  'id': '63OQupATfueTdZMWTxW03A',
  'uri': 'spotify:track:63OQupATfueTdZMWTxW03A',
  'track_href': 'https://api.spotify.com/v1/tracks/63OQupATfueTdZMWTxW03A',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/63OQupATfueTdZMWTxW03A',
  'duration_ms': 264067,
  'time_signature': 4}]

<h4>Building an Audio Feature DataFrame</h4>
Finally, if desired we can use the result above to make a nice dataframe for analyses and visuals.

In [4]:
import pandas as pd

# Dropping a few unnecessary columns for readability
audio_features_df = pd.DataFrame(audio_features).drop(['id', 'uri', 'type', 'track_href', 'analysis_url'], axis = 1)

audio_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.36,0.501,7,-9.129,1,0.0258,0.0638,0.000093,0.172,0.324,74.807,264067,4


From here we can easily add the artist and song name to the dataframe using the <b>track_artist</b> and <b>track_name</b> variables from first step.

In [5]:
# Add previously stored artist name
audio_features_df.insert(0, 'artist', track_artist)

# Add previously stored track name
audio_features_df.insert(1, 'song_title', track_name)

# Final result
audio_features_df

,artist,song_title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Radiohead,Karma Police,0.36,0.501,7,-9.129,1,0.0258,0.0638,0.000093,0.172,0.324,74.807,264067,4


<br>
With that we have a template by which we can build out a dataframe or dataframes! Again, we're not using this on this particular project, but it is a fun and easy share.

---

<h2>Audio Analysis</h2>
By comparison, the audio analysis provides a breakdown of a song's <i>rhythm</i>, <i>pitch</i>, and <i>timbre</i>. <br>
<br>
For this project, we will instead be using Spotify's audio analysis. According to <a href ='https://developer.spotify.com/documentation/web-api/reference/get-audio-analysis'>Spotify Developer's Site</a>, the audio analysis describes a track's musical structure, including rhythm, pitch, and <u><b>timbre</i></u>. Let's look closer at the latter two:<br>


In [39]:
track_analysis_df = pd.DataFrame(sp.audio_analysis('spotify:track:63OQupATfueTdZMWTxW03A')['segments'])

track_analysis_df.head()

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,pitches,timbre
0,0.00000,0.11134,0.000,-60.000,0.00000,-60.000,0.0,"[1.0, 0.725, 0.337, 0.371, 0.431, 0.408, 0.569...","[0.05, 169.954, 8.395, -29.988, 56.787, -50.22..."
1,0.11134,0.84522,1.000,-60.000,0.05445,-19.323,0.0,"[0.038, 0.03, 0.006, 0.01, 0.101, 0.013, 0.034...","[33.644, -23.384, -74.375, 34.627, 18.404, 63...."
2,0.95655,0.57519,1.000,-34.527,0.04297,-15.601,0.0,"[0.099, 0.042, 0.018, 0.017, 0.124, 0.028, 0.1...","[40.11, 5.728, -22.954, 30.254, 10.283, -4.349..."
3,1.53175,0.23787,0.696,-29.430,0.10532,-22.314,0.0,"[0.135, 0.064, 0.053, 0.038, 0.17, 0.044, 0.11...","[34.347, -6.78, 5.479, -62.715, 3.666, -8.412,..."
4,1.76961,0.44463,0.447,-26.881,0.03024,-20.798,0.0,"[0.032, 0.084, 0.02, 0.024, 0.056, 0.214, 1.0,...","[33.805, -53.703, -75.956, 43.627, -25.619, -4..."


<h3>Pitch</h3>
Imagine hitting a key on a piano. When you strike the key, the piano produces a sound wave which you perceive as the note played, say C. Now, imagine playing a different C key on the piano. It will sound the same but the note will be higher or lower depending on which C you played. That's pitch. Pitch is simply how high or low a note sounds. A higher pitch produces a sound wave with a shorter wavelength which results in a higher note while a lower pitch produces a sound wave with a longer wavelength resulting in a lower note.<br>
There are 12 pitch classes, or semitones, in Western Music: C, C#, D, D#, E, F, F#, G, G#, A, A#, B. When we examine the individual song segments of the audio analysis, each one of these semitones is evaluated on a scale of 0 to 1 which represents how prevalent the pitch is in that segment. Let's use the audio analysis of our song as an example.

<h4>Examine the pitches of the first 5 segments</h4>
We will need to use this again later on, so we will create a function for this method.

In [63]:
def create_pitch_df(uri):

    # Create the audio analysis dataframe
    audio_analysis_df = pd.DataFrame(sp.audio_analysis(uri)['segments'])
    
    # Create a list of the 12 pitch classes
    semitones = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

    # Create a pitch dataframe 
    pitch_df = audio_analysis_df[['duration', 'pitches']].copy()

    # Expand the pitches and label by class
    for i in range(12):
        pitch_df[semitones[i]] = pitch_df['pitches'].apply(lambda x: x[i])

    # Drop the pitches column
    pitch_df.drop(['pitches'], axis = 1, inplace = True)

    return pitch_df


# Radiohead Karma Police Pitch DF
rh_kp_pitch_df = create_pitch_df(track_uri)

rh_kp_pitch_df.head(5)

,duration,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
0,0.11134,1.000,0.725,0.337,0.371,0.431,0.408,0.569,0.603,0.690,0.674,0.686,0.120
1,0.84522,0.038,0.030,0.006,0.010,0.101,0.013,0.034,0.048,0.031,1.000,0.042,0.016
2,0.57519,0.099,0.042,0.018,0.017,0.124,0.028,0.121,0.221,0.070,1.000,0.063,0.053
3,0.23787,0.135,0.064,0.053,0.038,0.170,0.044,0.116,0.127,0.149,1.000,0.248,0.035
4,0.44463,0.032,0.084,0.020,0.024,0.056,0.214,1.000,0.139,0.029,0.032,0.022,0.013


According to the documentation, pitches are normalized to 1 by their strongest dimension. Looking at the dataframe above we see the prevalent first 5 notes are C, A, A, A, F#.<br>
<br>
Like the audio features, I've seen a ton of work done with this data. However, it still doesn't quite fit what we're after. <i><b>Why?</b></i> Even though they can produce the same pitch, a note played on a piano will obviously sound different than the same note played on, say, a guitar. What element creates that distinction? <b>Timbre</b>.

<h3>Timbre</h3>
Timbre (pronounced tam-ber) is the color of music. It describes the tone color and tone quality of music. I personally like to describe it as how a song feels. The best way to think about timbre is to imagine two people, a man and a woman, singing the <i>exact</i> same notes at the exact same pitch. They will produce all the same musical attributes, but they will obviously not sound the same. That's timbre.<br>
I chose timbre for this project because to me this is why we prefer particular songs to others. Rhythm may play some part in it as some people may prefer slow songs to more upbeat songs or maybe even prefer some keys to others, but there are still distinctions between the slow songs in the same key and that distinction lies in the timbre. 
Looking at a 3D model of timbre, then, means we are looking at how a song <i>feels</i>. That's exactly what we are after.<br>
Furthermore, it will be timbre that will help us find songs that sound like one another when we build our deep learning model. Look no further than the <a href='https://developer.spotify.com/documentation/web-api/reference/get-audio-analysis'>audio analysis documentation on Spotify's Developer Site</a> on timbre for verification of this: <i>"Timbre vectors are best used in comparison with each another."</i>

<h4>A quick look at timbre values</h4>
We will see a lot more of this in the next notebook, <a href='https://nbviewer.org/github/JonYarber/music_modeling/blob/main/4.%20Creating%20a%203D%20Music%20Model.ipynb'>Creating a 3D Music Model</a>.

In [62]:
# Create the audio analysis dataframe
audio_analysis_df = pd.DataFrame(sp.audio_analysis(track_uri)['segments'])
    
# Create a pitch dataframe 
timbre_df = audio_analysis_df[['duration', 'timbre']].copy()

# Expand the timbres and label
for i in range(12):
    timbre_df[f'timbre_{i + 1}'] = timbre_df['timbre'].apply(lambda x: x[i])

# Drop the original timbre column
timbre_df.drop(['timbre'], axis = 1, inplace = True)

# Look at first 5 timbre vectors
timbre_df.head()

,duration,timbre_1,timbre_2,timbre_3,timbre_4,timbre_5,timbre_6,timbre_7,timbre_8,timbre_9,timbre_10,timbre_11,timbre_12
0,0.11134,0.050,169.954,8.395,-29.988,56.787,-50.228,14.894,3.851,-27.467,0.920,-10.538,-6.641
1,0.84522,33.644,-23.384,-74.375,34.627,18.404,63.062,7.419,-30.435,-15.794,48.255,36.766,-8.482
2,0.57519,40.110,5.728,-22.954,30.254,10.283,-4.349,-4.717,-0.904,-1.862,-9.790,-17.182,5.178
3,0.23787,34.347,-6.780,5.479,-62.715,3.666,-8.412,-9.065,4.048,4.949,-0.829,-11.058,0.388
4,0.44463,33.805,-53.703,-75.956,43.627,-25.619,-46.175,7.318,-32.674,-14.825,25.307,-9.279,18.329


Now that we know what we are looking for and how to obtain it, we will move on to the next step where I will show you how to create the 3D timbre model.